# Script #1 

- Takes in Sharepoint Export `OTS_RSVP_45.xlsx`
- Renames column headers to match the Knack Reservation schema `banners_reservations_ots.csv`
- Exports new Reservations table to `output/reservations.csv`
- Makes a new Reservations Locations Table to match schema from `banner_locations.csv`
- Exports new Reservations Locations to `output/reservation_locations.csv`

In [75]:
import pandas as pd
from pandas import read_excel
import numpy as np

df = pd.read_excel("./source_data/OTS_RSVP_45.xlsx", sheet_name="owssvr.dll")

In [76]:
# Setup a new dataframe for Reservations:
df_reservations = df

# Rename columns to match Knack schema
# Sharepoint: OTS_RSVP_45 column names of left
# Knack: banner_reservations_ots on right
column_names = {
    # Sharepoint : Knack
    'ID': 'Sharepoint ID',
    'Permit #': 'Permit Number',
    'EventName': 'Event Name OTS', 
    'Organization': 'Organization Name', 
    # 'Payment Rec?'
    'Delivered?': 'Banners Delivered',
    'Status': 'Banner Status OTS',
    'Quantity': 'Number of Banners OTS',
    # Count Banner Jobs
    'RSVP Date': 'CREATED_DATE',
    'Installation_Date': 'Installation Date OTS',
    'Removal_Date': 'Removal Date OTS',
    # Keep all 'Banner_Face' & 'OTS_Linker' 1-4 columns the same
    # 'App Received?'
    # 'Co-Sponsor?'
    # 'Design Rec?'
    'Will Pick-up?': 'Will Pick Up'
    # 'Item Type'
    # 'Path'
}
df_reservations.rename(columns = column_names, inplace= True)

# TODO: Confirm if we should we remove these unneeded columns?
removed_columns = [
#     'Payment Rec?',
#     'App Received?',
#     'Co-Sponsor?',
#     'Design Rec?',
    'Item Type',
    'Path'
]
df_reservations = df_reservations.drop(columns=removed_columns)



In [77]:
# Export Reservations table to CSV
df_reservations.to_csv(r'./output_data/reservations.csv', index=None, header=True)

# Script #2

### Inputs:
- Sharepoint reservations table `df_reservations`
- Reservation table `source_data/banner_reservations_ots_20190325.csv` w/ Knack given "Banner OTS ID"
- Reservation Location Table Schema `output_data/reservation_locations.csv`
- Location Table Schema `source_data/banners_over_the_street_locations.csv`

### Outputs:
- creates a new Reservation Locations Table
    - with a foreign key column of Knack ID of Reservation
    - with a "Location Name" column matching Location Table "Linker Label" as identifier 

In [80]:
# Import Reservation Table. We'll use this data to add "banner reservations OT" 
# data to the Reservation Locations dataframe using the "Banner OTD ID" column.
# 
# NOTE: banner_reservations_ots_20190325 had duplicate columns so I manually removed 
# the "Number of Banners OTS" col and resaved as "banner_reservations_ots_20190325_2.csv"
knack_reservations_df = pd.read_csv('./source_data/banner_reservations_ots_20190325_2.csv')

# Merge Banner OTS ID column to original Sharepoint Export
sharepoint_reservations_df = df_reservations

for i in sharepoint_reservations_df.index:
    # Give the Sharepoint rsvp Table Banner OTS IDs that match the Reservations Table exported from Knack
    banner_id = knack_reservations_df.loc[ knack_reservations_df["Sharepoint ID"] == sharepoint_reservations_df["Sharepoint ID"], 'Banner OTS ID']
    sharepoint_reservations_df["banner reservations OT"] = banner_id

In [81]:
# Create a new dataframe for Reservations Locations 
# using headers from the csv
reso_location_column_names = pd.read_csv('./source_data/banner_locations.csv').columns
reso_locations_df = pd.DataFrame(columns=reso_location_column_names)
reso_locations_df['Parent ID'] = None

In [83]:
locations_df = pd.read_csv('./source_data/banners_over_the_street_locations.csv')

In [90]:
import math

for i in df_reservations.index:
    has_one_location = df_reservations["Number of Banners OTS"][i] == 1
    

    
    if has_one_location:
        # Get Location value
        linker_location_series = locations_df.loc[ locations_df["Linker Name"] == df_reservations['OTS_Linker_1'][i], 'Linker Label']
        if len(linker_location_series) > 0: 
            linker_location = linker_location_series.values[0]
        else:
            linker_location = None
        
        # Change outdated LAMAR #8 address
        if df_reservations[f'OTS_Linker_1'][i] == '# 8: 4180 1/2 N LAMAR BLVD':
              linker_location = '# 8: 4199 1/2 N LAMAR BLVD (N - S)'
              linker_name = '# 8: 4199 1/2 N LAMAR BLVD'
        else:
            linker_name = df_reservations['OTS_Linker_1'][i]
        
        # TODO: Figure out why this one didn't take!
        # Change LAMAR #2 address
        if df_reservations[f'OTS_Linker_1'][i]  == '# 2: 9411 1/2 N LAMAR BLVD':
              linker_location = '#2: 9411 1/2 N LAMAR BLVD (N - S)'
              linker_name = '# 2: 9411 1/2 N LAMAR BLVD'
        else:
            linker_name = df_reservations[f'OTS_Linker_1'][i]
        
        data = {
            'Bbanner reservations OT': df_reservations["banner reservations OT"][i],
            'Permit Number': df_reservations["Permit Number"][i],
            'banners_organization': df_reservations["Organization Name"][i],
            'Organization Name': df_reservations['Organization Name'][i],
            'Linker Location': linker_location,
            'Linker Name': df_reservations[f'OTS_Linker_1'][i],
            'Direction Facing': df_reservations['Banner_Face_1'][i],
            'OTS Install Date ': df_reservations['Installation Date OTS'][i],
            'OTS Remove Date': df_reservations['Removal Date OTS'][i],
            'Count Total': df_reservations['Number of Banners OTS'][i],
            'Status OTS': df_reservations['Banner Status OTS'][i],
        }
    
        # Create a new row
        row_series = pd.Series(data)
        # Add row to dataframe at the current iterator index
        reso_locations_df.loc[i] = row_series

    else:        
        x = 0
        while x < df_reservations["Number of Banners OTS"][i]:
            x += 1
            
            # This if is for an edge case were the row doesn't have a Permit Numbers
            # This seems to happen when the org for a banner is ATD.
            if type(df_reservations["Permit Number"][i]) == float and math.isnan(df_reservations["Permit Number"][i]):
                permit_number = None
                parent_id = None
                
            else:
                parent_id = df_reservations["Permit Number"][i].split(" Permits ")[0].split("Parent ID")[1].strip().rstrip(',')
                permit_list = df_reservations["Permit Number"][i].split(" Permits ")[1].split(",")
                
                permit_number = permit_list[x - 1].strip()
            
            # Get Location value
            linker_location_series = locations_df.loc[ locations_df["Linker Name"] == df_reservations[f'OTS_Linker_{x}'][i], 'Linker Label']
                  
            if len(linker_location_series) > 0: 
                linker_location = linker_location_series.values[0]
            else:
                linker_location = None
            
            # Change outdated LAMAR #8 address
            if df_reservations[f'OTS_Linker_{x}'][i] == '# 8: 4180 1/2 N LAMAR BLVD':
                  linker_location = '# 8: 4199 1/2 N LAMAR BLVD (N - S)'
                  linker_name = '# 8: 4199 1/2 N LAMAR BLVD'
            else:
                linker_name = df_reservations[f'OTS_Linker_{x}'][i]
            
            # Change LAMAR #2 address
            if df_reservations[f'OTS_Linker_{x}'][i]  == '# 2: 9411 1/2 N LAMAR BLVD':
                  linker_location = '#2: 9411 1/2 N LAMAR BLVD (N - S)'
                  linker_name = '# 2: 9411 1/2 N LAMAR BLVD'
            else:
                linker_name = df_reservations[f'OTS_Linker_{x}'][i]
                  
        
            data = {
                'banner reservations OT': df_reservations["banner reservations OT"][i],
                'Permit Number': permit_number,
                'banners_organization': df_reservations["Organization Name"][i],
                'Organization Name': df_reservations['Organization Name'][i],
                'Linker Location': linker_location,
                'Linker Name': linker_name,
                'Direction Facing': df_reservations[f'Banner_Face_{x}'][i],
                'OTS Install Date ': df_reservations['Installation Date OTS'][i],
                'OTS Remove Date': df_reservations['Removal Date OTS'][i],
                'Count Total': df_reservations['Number of Banners OTS'][i],
                'Status OTS': df_reservations['Banner Status OTS'][i],
                'Parent ID': parent_id
            }
            

            # Create a new row
            row_series = pd.Series(data)
            # Add row to dataframe at the current iterator index
            reso_locations_df.loc[f'{i}0{x}'] = row_series               

In [91]:
# Export Reservations Locations table to CSV
reso_locations_df.to_csv(r'./output_data/reservation_locations.csv', index=None, header=True)